<a href="https://colab.research.google.com/github/Caroline-Camargo/TrabalhosFerramentasDeInteligenciaArtificial/blob/main/Tarefa1_Engenharia_De_Atributos_Caroline_Souza_Camargo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Engineering with Sklearn - Descrição da Tarefa

1.   Item da lista
2.   Item da lista




Baixe os três conjuntos de dados. Cada um deles possui características de atributos distintas. Utilizando técnicas de engenharia de atributos com sklearn (ou outra ferramenta), selecione a menor quantidade de atributos possível em cada um deles, mas sem remover atributos significativos.

## Download Datasets

In [ ]:
!pip install gdown -qqq

In [ ]:
!gdown https://drive.google.com/uc?id=1q_VpqJ8O8JazQez-N6SYTBb51T4QrpMz
!gdown https://drive.google.com/uc?id=1tFcc2B7UfIzy7q3P49EzVxgetdRnbxfc
!gdown https://drive.google.com/uc?id=13fGg8YT844EN3xLgl5UzW1HOsOIbqYPo

Downloading...
From: https://drive.google.com/uc?id=1q_VpqJ8O8JazQez-N6SYTBb51T4QrpMz
To: /content/ia_fe_dataset1.csv
100% 161k/161k [00:00<00:00, 93.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tFcc2B7UfIzy7q3P49EzVxgetdRnbxfc
To: /content/ia_fe_dataset2.csv
100% 1.55M/1.55M [00:00<00:00, 160MB/s]
Downloading...
From: https://drive.google.com/uc?id=13fGg8YT844EN3xLgl5UzW1HOsOIbqYPo
To: /content/ia_fe_dataset3.csv
100% 3.08M/3.08M [00:00<00:00, 166MB/s]


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector as SFS


#DATASETS


##Dataset1


In [ ]:
import pandas as pd
import numpy as np

dataSet = pd.read_csv('/content/ia_fe_dataset1.csv')
dataSet1 = dataSet.copy()

corr_matrix = dataSet1.corr()
corr_matrix.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,target
feature_1,1.000000,0.088525,0.088525,0.985768,-0.002907,-0.662452
feature_2,0.088525,1.000000,1.000000,0.254715,-0.017930,0.012852
feature_3,0.088525,1.000000,1.000000,0.254715,-0.017930,0.012852
feature_4,0.985768,0.254715,0.254715,1.000000,-0.005849,-0.640958
feature_5,-0.002907,-0.017930,-0.017930,-0.005849,1.000000,0.004133


Analizando a tabela de correlação é possível perceber que é necessário realizar o pré-processamento dos dados. Observa-se que existem features que têm alta correlação entre si, remover uma das features com alto grau de correlação é necessário por que é considerado uma informação redundante, pois não adiciona novas informações ao modelo. Além disso elas aumentam desnecessariamente a dimensionalidade dos dados e dificultam a interpretação.
O critério adotado para remoção de features foi o valor 0.85, onde se considerou que a partir desse valor as features estariam altamente correlacionadas.

In [ ]:
def remocaoFeaturesAltaCorrelacao(dataSet, correlacao=0.85):
  corr_matrix = dataSet.corr().abs()

  columns = corr_matrix.columns
  dropped_features = set()

  for i in range(len(corr_matrix)):
    for j in range(i + 1, len(corr_matrix.columns)):
      if corr_matrix.iloc[i, j] > correlacao:
        feature = columns[j]
        dropped_features.add(feature)

  dataSet.drop(columns=dropped_features, inplace=True)
  return dataSet

dataSet1 = remocaoFeaturesAltaCorrelacao(dataSet1)
corr_matrix = dataSet1.corr()
corr_matrix.head()

,feature_1,feature_2,feature_5,target
feature_1,1.000000,0.088525,-0.002907,-0.662452
feature_2,0.088525,1.000000,-0.017930,0.012852
feature_5,-0.002907,-0.017930,1.000000,0.004133
target,-0.662452,0.012852,0.004133,1.000000


Além disso, é também necessário remover features irrelevantes, ou seja, que tem baixa correlação com o target. Uma vez que concentra a atenção do modelo apenas nas informações mais relevantes, resultando em maior precisão e generalização.
O critério adotoado para remoção de features foi o valor 0.00, qualquer feature abaixo desse valor foi removida

In [ ]:
def remocaoFeaturesBaixaCorrelacaoTarget(dataSet, correlacao=0.01):
    corr_matrix = dataSet.corr().abs()
    target_corr = corr_matrix['target']
    features_to_drop = target_corr[target_corr < correlacao].index
    dataSet.drop(columns=features_to_drop, inplace=True)
    return dataSet

dataSet1 = remocaoFeaturesBaixaCorrelacaoTarget(dataSet1)
corr_matrix = dataSet1.corr()
corr_matrix.head()

,feature_1,feature_2,target
feature_1,1.000000,0.088525,-0.662452
feature_2,0.088525,1.000000,0.012852
target,-0.662452,0.012852,1.000000


Por fim, é obtido como resultado um conjunto de dados refinado e melhorado, com features relevantes, sem features redundantes e devidamente processadas para capturar os padrões e informações importantes dos dados.

Para a escolha dos targets, além da correlação, pode ser utilizado outras técnicas, como o aprendizado de máquina usando RandomForestClassifier:

In [ ]:
def select_From_Model(dataSet):
  x = dataSet.drop(columns=['target'])
  y = dataSet['target']

  clf = RandomForestClassifier(n_estimators=100, random_state=42)
  clf.fit(x, y)

  sfm = SelectFromModel(clf, prefit=True)
  selected_features = sfm.get_support()

  selected_features = x.columns[selected_features]
  print('\nFeatures Escolhidas: ')
  print(', '.join(selected_features))

dataSet1 = dataSet.copy()
select_From_Model(dataSet1)


Features Escolhidas: 
feature_1, feature_4


É possível observar que o modelo escolheu features com alto grau de correlação entre si, que foram removidas quando fiz a seleção por alto grau de correlação.

Acredito que uma abordagem combinada seria o mais interessante nesse caso. Onde a seleção por correlação poderia ser aplicada como uma etapa inicial de pré-processamento para remover features altamente correlacionadas e, em seguida, aplicar a seleção com Random Forest para capturar relações não-lineares e identificar as features mais importantes para a tarefa de classificação




Técnicas ainda mais eficazes para identificar as combinações ideais de features podem ser usadas como Eliminação Recursiva de Features e a Seleção Sequencial de Features. A primeira é uma técnica de seleção de características que funciona removendo recursivamente as features menos importantes de um modelo até que o número desejado de features seja alcançado e a segunda técnica começa com um conjunto vazio de features e adiciona iterativamente a feature que resulta no maior ganho de desempenho do modelo.

In [ ]:
def recursiveFeatureElimination(dataSet, n=2):
  x = dataSet.drop(columns=['target'])
  y = dataSet['target']

  clf = RandomForestClassifier(n_estimators=100, random_state=42)
  rfe = RFE(estimator=clf, n_features_to_select=n, step=1)
  rfe.fit(x, y)
  selected_features = rfe.get_support()

  selected_features = x.columns[selected_features]
  print('\nFeatures Escolhidas: ')
  print(', '.join(selected_features))
  dataSet = dataSet[list(selected_features) + ['target']]

dataSet1 = dataSet.copy()
recursiveFeatureElimination(dataSet1)


Features Escolhidas: 
feature_1, feature_4


In [ ]:
def sequentialFeatureSelector(dataSet, n=2):
  x = dataSet.drop(columns=['target'])
  y = dataSet['target']

  clf = RandomForestClassifier(n_estimators=100)
  sfs = SFS(clf, n_features_to_select=n)
  sfs = sfs.fit(x, y)
  selected_features = sfs.get_support()

  selected_features = x.columns[selected_features]
  print('\nFeatures Escolhidas: ')
  print(', '.join(selected_features))
  dataSet = dataSet[list(selected_features)   + ['target']]

dataSet1 = dataSet.copy()
sequentialFeatureSelector(dataSet1)


Features Escolhidas: 
feature_1, feature_2


##Dataset2



Para os dataSets seguintes utilizei a abordagem combinada, removendo features altamente correlacionadas e irrelevantes, em seguida, apliquei as técnicas para diferentes configurações dos modelos



In [ ]:
dataSet = pd.read_csv('/content/ia_fe_dataset2.csv')
dataSet2 = dataSet.copy()
dataSet2 = remocaoFeaturesAltaCorrelacao(dataSet2)
dataSet2 = remocaoFeaturesBaixaCorrelacaoTarget(dataSet2)

print("--- Select From Model ---")
select_From_Model(dataSet2.copy())

print("\n\n--- Recursive Feature Elimination ---")
recursiveFeatureElimination(dataSet2.copy(), 10)

print("\n\n--- Select From Model ---")
sequentialFeatureSelector(dataSet2.copy(), 5)


--- Select From Model ---

Features Escolhidas: 
feature_2, feature_3, feature_9, feature_13


--- Recursive Feature Elimination ---

Features Escolhidas: 
feature_2, feature_3, feature_4, feature_6, feature_9, feature_12, feature_13, feature_25, feature_29, feature_49


--- Select From Model ---

Features Escolhidas: 
feature_2, feature_3, feature_4, feature_13, feature_25


##Dataset3

In [ ]:
dataSet = pd.read_csv('/content/ia_fe_dataset3.csv')
dataSet3 = dataSet.copy()
dataSet3 = remocaoFeaturesAltaCorrelacao(dataSet3)
dataSet3 = remocaoFeaturesBaixaCorrelacaoTarget(dataSet3)


print("--- Select From Model ---")
select_From_Model(dataSet3.copy())

print("\n\n--- Recursive Feature Elimination ---")
recursiveFeatureElimination(dataSet3.copy(), 3)

print("\n\n--- Select From Model ---")
sequentialFeatureSelector(dataSet3.copy(), 4)


--- Select From Model ---

Features Escolhidas: 
feature_9, feature_10, feature_13, feature_14, feature_19, feature_20, feature_24, feature_27, feature_39, feature_43, feature_49, feature_62, feature_63, feature_65, feature_85, feature_88, feature_94, feature_98, feature_99


--- Recursive Feature Elimination ---

Features Escolhidas: 
feature_24, feature_27, feature_88


--- Select From Model ---

Features Escolhidas: 
feature_4, feature_20, feature_24, feature_85
